# Titanic Stacking
By Luis andrade

## Context
* Stacking is the process by which several estimators are leveraged in order to produced a combined prediction that achieves the best performance
* Stacking has been shown to perform well in a large variety of data science problems

## Objectives
* To implement a stacking of several well-known shallow estimators
* To apply the stacking result to the Titanick survival problem and evaluate its performance


## Dataset

The Dataset is the famous Titanick survival data, which contains information about the passengers that were onboard of the titanick. This dataset also includes whether the passenger survived or not, which in this case will be considered the target variable. 

The dataset contains hundreds of passengers with a total of 12 fields

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
import sklearn
import warnings
warnings.simplefilter("ignore")

## Part 1 - EDA

In this section some explanatory data analysis is carried out

In [ ]:
df = pd.read_csv("../input/train.csv")
df.head(10)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud()
wordcloud.generate(" ".join(df["Name"]))

fig = plt.figure(figsize=(40, 30))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## Part 2- Feature Engineering

We have briefly examined our dataset, so let us know do some feature engineering to extract relevant features

In [ ]:
def get_title(text):
    if "Mr" in text:
        return 2
    elif "Miss" in text:
        return 0
    elif "Mrs" in text:
        return 1
    else:
        return -1
    
def add_title_to_df(df_in):
    """Takes a Datarame df and adds a column title"""
    df = df_in.copy()
    df["title"] = df["Name"].apply(get_title)
    df.loc[:, "title"]
    return df

df_title = add_title_to_df(df)
df_title.head()

In [ ]:
def enrich_df(df_in):
    """Add columns sex_num, embarked_num and non-NAN Age to df"""
    df = df_in.copy()
    df["sex_num"] = df["Sex"].map({"male":1, "female":0})
    df["embarked_num"] = df["Embarked"].factorize()[0]
    df["Age"] = df["Age"].fillna(df["Age"].mean())
    return df

In [ ]:
# It seems that out of the 12 columns there are 7 numeric 
df_enriched = enrich_df(df_title)
df_enriched.describe()

In [ ]:
import seaborn as sns
correlation = df_enriched.drop("PassengerId", axis=1).corr().apply(abs)
mask = np.zeros_like(correlation)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(correlation, mask=mask, annot=True)

Looking at the correlation heatmap, it seems that Sex, Pclass, title, and Fare are the most correlated with the target variable

In [ ]:
correlation.loc["Survived"].sort_values(ascending=False)

In [ ]:
cols = correlation.columns.drop("Survived")
cols

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()

def df_to_X_and_y(df_in, cols, y_col=None):
    """Get X and y (if exists) from df"""
    df = df_in.copy()
    if y_col is not None:
        y = df.loc[:, y_col]
        X = df.drop(y_col, axis=1)[cols]
    else:
        y = None
        X = df[cols]
        
    X_new = pd.DataFrame(imputer.fit_transform(X),
                     index=X.index, columns=X.columns)  
    # The imputer is an imputation transformer for missing values
    
    return X_new, y

X, y = df_to_X_and_y(df_enriched, cols, "Survived")
X.head()

In [ ]:
y.head()

So we have our feature variables contained in matrix **X** and the target variable contained in vector $y$

### Part 2b - Dimensionality Reduction 

Since we have our feature and target variables, let us do some dimensionality reduction to create new features 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X)
plt.plot(range(1, X.shape[1]+1), pca.explained_variance_ratio_)
plt.title("Explained Variance Ratio Vs N components")
plt.show()


As can be seen by the above figure, the eplained variance greatly decreases after the third principal component. We can use this "elbow" to determine the number of principal components that we will use

In [ ]:
Z = PCA(n_components =3).fit_transform(X)
Z

In [ ]:
Z.shape

## Part 3 - Modeling

Since we now have our features and target ready, we can start exploring with some models

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline

scaler = StandardScaler()

def compare_classifiers(cls_list, cv=5, params=None):
    """
    This function compares different estimators
    
    Arguments
    ---------
    cls_list : list
        list of estimator classes
    cv : int, default=5
        Number of folds to use in cross-validation
    params : dict, default=None
        Dict indexed by class name whose values are dicts
        with the parameter values to feed to GridSearchCV
        By default, no gridsearch is performed
        
    Returns
    -------
    df : Pandas DataFrame
        Datarame containing the results of the estimators
    """
    scores_dict ={}
    for cls_ in cls_list:
        if params is not None:
            clf = cls_(n_estimators = params)
        else:
            clf = cls_()
        scores = {}
        pipeline = Pipeline([("scaler", scaler), (cls_.__name__, clf)])
        plain_score = cross_validate(clf, X, y, cv=cv)
        scaled_X = cross_validate(pipeline, X, y, cv=cv)
        scaled_Z = cross_validate(pipeline, Z, y, cv=cv)
        scores["plain "] = plain_score
        scores["scaled_X "] = scaled_X
        scores["scaled_Z "] = scaled_Z
        scores_dict[cls_.__name__] = scores

    df = pd.DataFrame(scores_dict)
    return df.T

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

cls_list = [LogisticRegression, BernoulliNB, KNeighborsClassifier,
            SVC, DecisionTreeClassifier]

df = compare_classifiers(cls_list)
df

In [ ]:
def show_heatmap(df, col_name):
    df_scores = df.applymap(lambda x : np.mean(x[col_name]))
    sns.heatmap(df_scores, annot=True)

In [ ]:
def show_model_times(df):
    df_times = df.applymap(lambda x : sum(x["fit_time"]) + sum(x["score_time"]))
    sns.heatmap(df_times, annot=True)

In [ ]:
from sklearn.ensemble import(AdaBoostClassifier, 
                             GradientBoostingClassifier,
                             BaggingClassifier,
                             ExtraTreesClassifier,
                             RandomForestClassifier)

ensemble_cls_list = [AdaBoostClassifier, 
                     GradientBoostingClassifier,
                     BaggingClassifier,
                     ExtraTreesClassifier,
                     RandomForestClassifier]

ensemble_df = compare_classifiers(ensemble_cls_list, cv=5, params=150)
ensemble_df

In [ ]:
concat_df = pd.concat([df, ensemble_df])
concat_df

In [ ]:
show_heatmap(concat_df, "test_score")

In [ ]:
show_model_times(concat_df)

### GridSearchCV

Now let us use grid search CV to find the best parameters for each of the estimators

In [ ]:
from sklearn.model_selection import GridSearchCV

def compare_gridsearch_classifiers(cls_list, X, y, cv=5, params=None, verbose=0):
    """
    This function compares different estimators
    
    Arguments
    ---------
    cls_list : list
        list of estimator classes
    X : pandas DataFrame
        Feature matrix
    y : pandas Series or array
        Target vector
    cv : int, default=5
        Number of folds to use in cross-validation
    params : dict, default=None
        Dict indexed by class name whose values are dicts
        with the parameter values to feed to GridSearchCV
        By default, no gridsearch is performed
    Verbose : int, default=0
        Control Verbose level
        
    Returns
    -------
    df : Pandas DataFrame
        Datarame containing the results of the estimators
    """
    scores_dict ={}
    best_params_dict={}
    for cls_ in cls_list:
        print(f"testing {cls_}")
        clf = cls_()
        scores = {}
        clf_name = cls_.__name__
        clf_params = params.get(clf_name)
        if clf_params is not None:
            pipeline = Pipeline([("scaler", scaler), (clf_name, clf)])
            gridsearch_plain = GridSearchCV(clf, clf_params, cv=cv, verbose=verbose)
            clf_params = {"__".join([clf_name, key]):value for key, value in clf_params.items()}
            gridsearch_scaled = GridSearchCV(pipeline, clf_params, cv=cv, verbose=verbose)
            scores["plain"] = gridsearch_plain.fit(X,y)
            scores["scaled_X "] = gridsearch_scaled.fit(X,y)
            #scores["scaled_Z "] = gridsearch_scaled.fit(Z, y).score(Z, y)
            scores_dict[clf_name] = scores

    return scores_dict

In [ ]:
ensemble_cls_list 

In [ ]:
standard_ensemble_dict = {"n_estimators":np.linspace(50, 300, 6, dtype=np.int16),
                         "learning_rate": np.linspace(0.01, 2, 5)}
parameters = {
    LogisticRegression.__name__:{"tol":[1e-3, 1e-4], "C": [0.1, 0.5, 1.0]},
    BernoulliNB.__name__:{"alpha": [0.1, 0.5, 1], "binarize": [0.0, 0.5], "fit_prior": [True, False]},
    KNeighborsClassifier.__name__:{"n_neighbors": [1, 3, 5, 10], "weights": ["uniform", "distance"]},
    SVC.__name__:{"C": [0.1, 0.5, 1.0], "kernel": ["linear", "rbf", "sigmoid"]},
    DecisionTreeClassifier.__name__:{"criterion": ["gini", "entropy"], "splitter":["best", "random"]},
    AdaBoostClassifier.__name__:{**standard_ensemble_dict},
    GradientBoostingClassifier.__name__:{**standard_ensemble_dict},
    BaggingClassifier.__name__:{},
    ExtraTreesClassifier.__name__:{"n_estimators": [100]},
    }

parameters

In [ ]:
scores_dict = compare_gridsearch_classifiers(cls_list, X, y, cv=5, params=parameters, verbose=2)

In [ ]:
ensemble_scores_dict = compare_gridsearch_classifiers(ensemble_cls_list, X, y, cv=5, params=parameters, verbose=2)

In [ ]:
scores_df = pd.DataFrame({**scores_dict, **ensemble_scores_dict})
scores_df.T 

In [ ]:
best_scores_df = scores_df.applymap(lambda x: x.best_score_)
sns.heatmap(best_scores_df.T, annot=True)

It seems that some estimators have a dev set performance of up to 83%, such as the SVC and the Gradient Boosting Classifier

In [ ]:
scores_df.index

In [ ]:
scaled_X_objs = scores_df.loc["scaled_X ",:]
preds = scaled_X_objs.apply(lambda x: x.best_estimator_.predict(X))

In [ ]:
preds_corr_df = pd.DataFrame.from_items(zip(preds.index, preds.values)).corr()
sns.heatmap(preds_corr_df, annot=True)

Since we are considering to use stacking it is also usefull to look at the correlation between the predictions of the different estimators. We want to stack models that whose predictions are not too correlated, otherwise the stacking may end up being redundant. Looking at the heatmap there seems to be two groups of "similar" estimators: the tree emsemble group and the remaining estimators. It would be interesting to use at least one estimator from each group in the stacking

## Part 4 - Stacking

We have looked and compared several estimators and tuned their hyperparameters, but can we get a better performance by using stacking?
Let's find out

The approach chosen here is to have a two-layer stacking, with three classifiers on the first layer and a "meta" classifier on the second layer

In [ ]:
from mlxtend.classifier import StackingCVClassifier # Library to help with the stacking

In [ ]:
# Let us chose the best tree emsemble estimator, the best non tree-ensemble estimator, and a third estimator (just to break the tie)
clf1= scaled_X_objs[SVC.__name__].best_estimator_ # best non tree-ensemble estimator
clf2= scaled_X_objs[GradientBoostingClassifier.__name__].best_estimator_ # best tree-ensemble estimator
clf3= scaled_X_objs[BernoulliNB.__name__].best_estimator_# third estimator to break the tie

In [ ]:
from sklearn.model_selection import cross_val_score
stacking_scores = {}

# Do the actual stacking, iterating through a list of metaclassifiers
for lr in [*ensemble_cls_list, *cls_list]:
    sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3],
                                meta_classifier=lr())
    stacking_scores[lr.__name__] = np.mean(cross_val_score(sclf, X.values, y.values))   

pd.Series(stacking_scores)

It seems that using the Random Forest Classifer as the meta classifier we manage to slightly outperform the single best model

## Part 5 - Making a Submission

Finnally, le us make a prediction to see how well our stacked classifier does on the leaderboard

In [ ]:
df_test = pd.read_csv("../input/test.csv")
X_test, _ = df_to_X_and_y(enrich_df(add_title_to_df(df_test)), cols, y_col=None)
final_clf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=SVC())
final_clf.fit(X.values, y.values)
preds = final_clf.predict(X_test.values)

In [ ]:
submission_df = pd.DataFrame({"Survived":preds }, index=df_test["PassengerId"])
submission_df.to_csv("submission.csv")
submission_df.head()

## Part 6 - Conclusion

To recap, this notebook started with a simple EDA to explore the dataset. Then, feature engineering was carried out to produce the desired variables. After this, a set of estimators were hyperparameter-tuned and and their performance was compared. Afterwards, a set of these estimators were used in the first layer of the stack along with a metaclassifier on the second layer. The best performance was obtained by one of the stacked models